# Understanding Session 2: Customization and Rate-Limited Concurrency

The `Session` object can be fully customized, including models, model parameters and rate limits, to accustom various usecases. 

Most usefully you can customize: 
- `llmconfig`: the default model parameters for every API call in the session
- `api_service`: rate limit api_service

## 1. Default llmconfig

In [1]:
from lionagi.llm_configs import oai_llmconfig
oai_llmconfig

{'model': 'gpt-4-1106-preview',
 'frequency_penalty': 0,
 'max_tokens': None,
 'n': 1,
 'presence_penalty': 0,
 'response_format': {'type': 'text'},
 'seed': None,
 'stop': None,
 'stream': False,
 'temperature': 0.7,
 'top_p': 1,
 'tools': None,
 'tool_choice': 'none',
 'user': None}

if you wish to change the default behavior
- you can either pass in a new llmconfig into the Session
- or update the config in the session directly

In [2]:
system = "you are a helpful assistant"

In [3]:
# passing in the llmconfig as a dict - recommended if want a completely different config or preset configuration
# llmconfig = {...}
# session1 = li.Session(system, 
#                       llmconfig=llmconfig)

In [4]:
import lionagi as li
# update the llmconfig directly in Session object - recommended if only a few changes
session2 = li.Session(system)
session2.llmconfig.update({"model": "gpt-3.5-turbo", "temperature": 0.5})

session2.llmconfig

{'model': 'gpt-3.5-turbo',
 'frequency_penalty': 0,
 'max_tokens': None,
 'n': 1,
 'presence_penalty': 0,
 'response_format': {'type': 'text'},
 'seed': None,
 'stop': None,
 'stream': False,
 'temperature': 0.5,
 'top_p': 1,
 'tools': None,
 'tool_choice': 'none',
 'user': None}

## 2. Default api_service

In [5]:
# import os
# from dotenv import load_dotenv
# load_dotenv()

# let's say you use more than one API key
# api_key2 = os.getenv("OPENAI_API_KEY2")

In [6]:
from lionagi.services import OpenAIService

# let us check the OpenAI api service
service = OpenAIService(
    # api_key = api_key2,           # you can change the api key here - default to OPENAI_API_KEY
    # token_encoding_name,          # or token encoding name  - default to OpenAI ChatCompletion
    max_requests_per_minute=10,     # or rate limits  - default to OpenAI tier-1 `gpt-4`
    max_tokens_per_minute=10_000
    )

# and then you can pass in the api_service to the session
session3 = li.Session(system, service=service)

In [7]:
# if you wish the rate limit to be applied across sessions, you need to pass in the same api_service when creating the sessions

session4 = li.Session(system, service=service)
session5 = li.Session(system, service=service)
session6 = li.Session(system, service=service)

# now the rate limit is applied across session3-6

## 3. Concurrency

In [8]:
# we will use numpy to generate random numbers for this part
# %pip install numpy

In [9]:
# let us use a simple conditional calculator session as an example
# in this example, we will have two steps in the instruction, first step would be choosing between sum or diff based on a case number
# and second step would be choosing between times or plus based on the sign of the first step

system = "You are asked to perform as a calculator. Return only a numeric value, i.e. int or float, no text."

instruct1 = {
    "sum the absolute values": "provided with 2 numbers, return the sum of their absolute values. i.e. |x|+|y|",}

instruct2 = {
    "diff the absolute values": "provided with 2 numbers, return the difference of absolute values. i.e. |x|-|y|",}

instruct3 = {
    "if previous response is positive": "times 2. i.e. *2", # case 0
    "else": "plus 2. i.e. +2",                              # case 1
}

In [10]:
# create a case and context
case = 0
context = {"x": 7, "y": 3}
instruct = instruct1 if case == 0 else instruct2

In [11]:
from timeit import default_timer as timer
start = timer()
calculator = li.Session(system, dir='data/logs/calculator/')

step1 = await calculator.initiate(instruct, context=context)
step2 = await calculator.followup(instruct3, temperature=0.5)     # you can also modify parameters for each API call

print(f"step1 result: {step1}")
print(f"step2 result: {step2}")

elapsed_time = timer() - start
print(f"run clock time: {elapsed_time:0.2f} seconds")

step1 result: 10
step2 result: 12
run clock time: 1.91 seconds


In [12]:
# now let us run 10 senerios in parallel
import numpy as np
num_iterations = 10

# generate random numbers
ints1 = np.random.randint(-10, 10, size=num_iterations)
ints2 = np.random.randint(0, 10, size=num_iterations)
cases = np.random.randint(0,2, size=num_iterations)

# let's define a simple parser function
f = lambda i: {"x": str(ints1[i]), "y": str(ints2[i]), "case": str(cases[i])}

# and create the various contexts, l_call (list call) is a helper function to simplify loop
contexts = li.l_call(range(num_iterations), f)

li.l_call(range(num_iterations), lambda i: print(contexts[i]));

{'x': '5', 'y': '5', 'case': '0'}
{'x': '-6', 'y': '8', 'case': '0'}
{'x': '-1', 'y': '7', 'case': '0'}
{'x': '-4', 'y': '5', 'case': '1'}
{'x': '8', 'y': '3', 'case': '1'}
{'x': '8', 'y': '6', 'case': '1'}
{'x': '-8', 'y': '0', 'case': '1'}
{'x': '6', 'y': '9', 'case': '1'}
{'x': '5', 'y': '1', 'case': '1'}
{'x': '-6', 'y': '0', 'case': '0'}


In [13]:
dir = 'data/logs/calculator'

In [14]:
# create a workflow for concurrent execution

async def calculator_workflow(context):
    
    calculator = li.Session(system, dir=dir)       # construct a session instance
    context = context.copy()
    case = int(context.pop("case"))
    
    instruct = instruct1 if case == 0 else instruct2
    await calculator.initiate(instruct, context=context)    # run the steps
    await calculator.followup(instruct3, temperature=0.5)
    
    calculator.messages_to_csv()        # log all messages to csv
    calculator.log_to_csv()             # log all api calls to csv
    return li.l_call(calculator.conversation.responses, lambda i: i['content'])

In [15]:
start = timer()

In [16]:
# use al_call (async list call) to run the workflow concurrently over all senerios
outs = await li.al_call(contexts, calculator_workflow)

5 logs saved to data/logs/calculator/messages_2023-12-28T00_46_36_496524.csv
2 logs saved to data/logs/calculator/llmlog_2023-12-28T00_46_36_497109.csv
5 logs saved to data/logs/calculator/messages_2023-12-28T00_46_36_816258.csv
2 logs saved to data/logs/calculator/llmlog_2023-12-28T00_46_36_816770.csv
5 logs saved to data/logs/calculator/messages_2023-12-28T00_46_36_898741.csv
2 logs saved to data/logs/calculator/llmlog_2023-12-28T00_46_36_899142.csv
5 logs saved to data/logs/calculator/messages_2023-12-28T00_46_36_922867.csv
2 logs saved to data/logs/calculator/llmlog_2023-12-28T00_46_36_923163.csv
5 logs saved to data/logs/calculator/messages_2023-12-28T00_46_37_102491.csv
2 logs saved to data/logs/calculator/llmlog_2023-12-28T00_46_37_103048.csv
5 logs saved to data/logs/calculator/messages_2023-12-28T00_46_37_104106.csv
2 logs saved to data/logs/calculator/llmlog_2023-12-28T00_46_37_104271.csv
5 logs saved to data/logs/calculator/messages_2023-12-28T00_46_37_104808.csv
2 logs save

In [17]:
elapsed_time = timer() - start
print(f"num_workload: {num_iterations}")
print(f"run clock time: {elapsed_time:0.2f} seconds")

num_workload: 10
run clock time: 2.07 seconds


In [18]:
for idx, out in enumerate(outs):
    print(f"Inputs: {ints1[idx]}, {ints2[idx]}, case: {cases[idx]}\n")
    print(f"Outputs: {out}")
    print("------\n")

Inputs: 5, 5, case: 0

Outputs: ['10.0', '10.0']
------

Inputs: -6, 8, case: 0

Outputs: ['14', '28']
------

Inputs: -1, 7, case: 0

Outputs: ['8', '16']
------

Inputs: -4, 5, case: 1

Outputs: ['1', '-3']
------

Inputs: 8, 3, case: 1

Outputs: ['5', '10']
------

Inputs: 8, 6, case: 1

Outputs: ['2', '4']
------

Inputs: -8, 0, case: 1

Outputs: ['8', '16']
------

Inputs: 6, 9, case: 1

Outputs: ['3', '5']
------

Inputs: 5, 1, case: 1

Outputs: ['4', '8']
------

Inputs: -6, 0, case: 0

Outputs: ['6.0', '2.0']
------



## 4. Customized api_service concurrent calls

by default, all the session will be created using the same default api_service to ensure rate limit is applied **globally**

But if you would like to have a different api_service and use across sessions, you need to pass in the **same** api_service object during construction

In [19]:
# now let us change the rate limit to check whether it is working
service = OpenAIService(max_requests_per_minute=10, max_tokens_per_minute=10_000)

async def calculator_workflow(context):
    
    calculator = li.Session(system, dir=dir, service=service)       # construct a session instance
    context = context.copy()
    case = int(context.pop("case"))
    instruct = instruct1 if case == 0 else instruct2

    await calculator.initiate(instruct, context=context)    # run the steps
    await calculator.followup(instruct3)
    
    calculator.messages_to_csv()        # log all messages to csv
    calculator.log_to_csv()             # log all api calls to csv

    return li.l_call(calculator.conversation.responses, lambda i: i['content'])

In [20]:
start = timer()

outs = await li.al_call(contexts, calculator_workflow)  

elapsed_time = timer() - start
print(f"num_workload: {num_iterations}")
print(f"run clock time: {elapsed_time:0.2f} seconds")

5 logs saved to data/logs/calculator/messages_2023-12-28T00_47_38_668920.csv
2 logs saved to data/logs/calculator/llmlog_2023-12-28T00_47_38_669254.csv
5 logs saved to data/logs/calculator/messages_2023-12-28T00_47_38_769745.csv
2 logs saved to data/logs/calculator/llmlog_2023-12-28T00_47_38_769954.csv
5 logs saved to data/logs/calculator/messages_2023-12-28T00_47_38_885544.csv
2 logs saved to data/logs/calculator/llmlog_2023-12-28T00_47_38_886015.csv
5 logs saved to data/logs/calculator/messages_2023-12-28T00_47_38_886430.csv
2 logs saved to data/logs/calculator/llmlog_2023-12-28T00_47_38_886568.csv
5 logs saved to data/logs/calculator/messages_2023-12-28T00_47_38_939701.csv
2 logs saved to data/logs/calculator/llmlog_2023-12-28T00_47_38_939995.csv
5 logs saved to data/logs/calculator/messages_2023-12-28T00_47_39_190912.csv
2 logs saved to data/logs/calculator/llmlog_2023-12-28T00_47_39_191339.csv
5 logs saved to data/logs/calculator/messages_2023-12-28T00_47_39_448392.csv
2 logs save

In [21]:
for idx, out in enumerate(outs):
    print(f"Inputs: {ints1[idx]}, {ints2[idx]}, case: {cases[idx]}\n")
    print(f"Outputs: {out}")
    print("------\n")

Inputs: 5, 5, case: 0

Outputs: ['10', '10']
------

Inputs: -6, 8, case: 0

Outputs: ['14', '28']
------

Inputs: -1, 7, case: 0

Outputs: ['6', '14']
------

Inputs: -4, 5, case: 1

Outputs: ['3', '-1']
------

Inputs: 8, 3, case: 1

Outputs: ['5', '10']
------

Inputs: 8, 6, case: 1

Outputs: ['2', '4']
------

Inputs: -8, 0, case: 1

Outputs: ['8', '16']
------

Inputs: 6, 9, case: 1

Outputs: ['3', '5']
------

Inputs: 5, 1, case: 1

Outputs: ['4', '8']
------

Inputs: -6, 0, case: 0

Outputs: ['6.0', '2.0']
------

